In [5]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import os, shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image
from PIL import ImageOps
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

data_dir="/Users/huangjie/ee541_super_resulotion/data"

# Load the data using the ImageFolder dataset class
batch_size=32

train_hr = os.path.join(data_dir,'training_hr')
train_lr = os.path.join(data_dir,'training_lr/X2')
val_hr = os.path.join(data_dir,'validation_hr')
val_lr = os.path.join(data_dir,'validation_lr/X2')
test_hr = os.path.join(data_dir,'test_hr')
test_lr = os.path.join(data_dir,'test_lr/X2')


In [6]:
#SKIP

#find smallest dimensions

image_files1 = [os.path.join(test_lr, file) for file in os.listdir(test_lr) if file.endswith('.png')]
image_files2 = [os.path.join(train_lr, file) for file in os.listdir(train_lr) if file.endswith('.png')]
image_files3 = [os.path.join(val_lr, file) for file in os.listdir(val_lr) if file.endswith('.png')]
image_files = image_files1 + image_files2 + image_files3

min_height = float('inf')
min_width = float('inf')
min_height_image = None
min_width_image = None

for image_path in image_files:
    image = Image.open(image_path)
    width, height = image.size
    if height < min_height:
        min_height = height
        min_height_image = image_path
    if width < min_width:
        min_width = width
        min_width_image = image_path


print("Image with smallest height:", min_height_image, "Height:", min_height)
print("Image with smallest width:", min_width_image, "Width:", min_width)
#ensuring that min height and min width will not round to be outside of range
min_height = min_height-1
min_width = min_width-1

Image with smallest height: /Users/huangjie/ee541_super_resulotion/data/training_lr/X2/0437x2.png Height: 324
Image with smallest width: /Users/huangjie/ee541_super_resulotion/data/training_lr/X2/0350x2.png Width: 558


In [7]:
#determine patch size
min_height = 36
min_width = 36

# Define the transformations to be applied to the images
#seed = 123
#generator = torch.Generator()
#generator.manual_seed(seed)
from torchvision.transforms.functional import rotate
from torchvision.transforms.functional import hflip
transform = {
    'train_input': transforms.Compose([
        transforms.CenterCrop([min_height,min_width]),
        # transforms.RandomApply([
        #   transforms.RandomHorizontalFlip(p=0.2),
        #   transforms.RandomVerticalFlip(p=0.2),
        #   transforms.RandomRotation(30),
        # ], p=0.5),
        transforms.ToTensor(),
    ]),
    'train_output': transforms.Compose([
        transforms.CenterCrop([min_height*2,min_width*2]),
        # transforms.RandomApply([
        #   transforms.RandomHorizontalFlip(p=0.2),
        #   transforms.RandomVerticalFlip(p=0.2),
        #   transforms.RandomRotation(30),
        # ], p=0.5),
        transforms.ToTensor(),
    ]),
    'val_test_input': transforms.Compose([
        transforms.CenterCrop([min_height,min_width]),
        transforms.ToTensor(),
    ]),
    'val_test_output': transforms.Compose([
        transforms.CenterCrop([min_height*2,min_width*2]),
        transforms.ToTensor(),
    ]),
    'val_test_input_rotate': transforms.Compose([
        transforms.Lambda(lambda x: rotate(x, 30)),
        transforms.CenterCrop([min_height,min_width]),
        transforms.ToTensor(),
    ]),
    'val_test_output_rotate': transforms.Compose([
        transforms.Lambda(lambda x: rotate(x, 30)),
        transforms.CenterCrop([min_height*2,min_width*2]),
        transforms.ToTensor(),
    ]),
    'train_input_rotate': transforms.Compose([
        transforms.Lambda(lambda x: rotate(x, 30)),
        transforms.CenterCrop([min_height,min_width]),
        transforms.ToTensor(),
    ]),
    'train_output_rotate': transforms.Compose([
        transforms.Lambda(lambda x: rotate(x, 30)),
        transforms.CenterCrop([min_height*2,min_width*2]),
        transforms.ToTensor(),
    ]),
    'train_input_flip': transforms.Compose([
        transforms.Lambda(lambda x: hflip(x)),
        transforms.CenterCrop([min_height,min_width]),
        transforms.ToTensor(),
    ]),
    'train_output_flip': transforms.Compose([
        transforms.Lambda(lambda x: hflip(x)),
        transforms.CenterCrop([min_height*2,min_width*2]),
        transforms.ToTensor(),
    ])
}

In [8]:
class Div2K(Dataset):
  def __init__(self, input_folder, output_folder, transform_input, transform_output):
    self.input_folder = input_folder
    self.output_folder = output_folder
    self.transform_input = transform_input
    self.transform_output = transform_output
    self.input_images = sorted([file for file in os.listdir(input_folder) if file.endswith('.png')])
    self.output_images = sorted([file for file in os.listdir(output_folder) if file.endswith('.png')])
    
  def __len__(self):
    return len(self.input_images)
  def __getitem__(self,index):
    input_image_path = os.path.join(self.input_folder, self.input_images[index])
    output_image_path = os.path.join(self.output_folder, self.output_images[index])
    input_image = Image.open(input_image_path).convert('RGB')
    output_image = Image.open(output_image_path).convert('RGB')

    input_image = self.transform_input(input_image)
    output_image = self.transform_output(output_image)
    return input_image, output_image

train_dataset = Div2K(train_lr, train_hr, transform_input=transform['train_input'],transform_output=transform['train_output'])
train_dataset_rotate = Div2K(train_lr, train_hr, transform_input=transform['train_input_rotate'],transform_output=transform['train_output_rotate'])
train_dataset_flip = Div2K(train_lr, train_hr, transform_input=transform['train_input_flip'],transform_output=transform['train_output_flip'])
test_dataset = Div2K(test_lr, test_hr, transform_input = transform['val_test_input'],transform_output=transform['val_test_output'])
val_dataset = Div2K(val_lr, val_hr, transform_input = transform['val_test_input'],transform_output=transform['val_test_output'])

temp = torch.utils.data.ConcatDataset([train_dataset, train_dataset_rotate])
train_dataset_combined = torch.utils.data.ConcatDataset([temp, train_dataset_flip])

batch_size = 24

train_dataloader_rotate = DataLoader(train_dataset_rotate, batch_size=batch_size, shuffle=True, num_workers=2)
train_dataloader_flip = DataLoader(train_dataset_flip, batch_size=batch_size, shuffle=True, num_workers=2)
train_dataloader_combined = DataLoader(train_dataset_combined, batch_size=batch_size, shuffle=True, num_workers=2)
#train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [9]:
#Chekcihng dataloader dimensions (opt)
data_iter = iter(train_dataloader_combined)
lr_images, hr_images = next(data_iter)
print("Low-resolution images:")
print("Shape:", lr_images.shape)
print("Data type:", lr_images.dtype)

print("\nHigh-resolution images:")
print("Shape:", hr_images.shape)
print("Data type:", hr_images.dtype)

#displaying some image pairs (opt)
i=0
for lr_images,hr_images in train_dataloader_combined:
    # Plot the first image in the batch
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    #expected shape (height, width, num_channels)
    lr_image = lr_images[0].permute(1, 2, 0)
    ax[0].imshow(lr_image)
    ax[0].set_title('Low-resolution')
    hr_image = hr_images[0].permute(1, 2, 0)
    ax[1].imshow(hr_image)
    ax[1].set_title('High-resolution')
    plt.show()
    i+=1
    if(i>2):
        break

PicklingError: Can't pickle <function <lambda> at 0x7f77d9e8f040>: attribute lookup <lambda> on __main__ failed

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, num_channels):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.prelu = nn.PReLU()
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.prelu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        return out

class UpsampleBlock(nn.Module):
    def __init__(self, num_channels):
        super(UpsampleBlock, self).__init__()

        self.conv = nn.Conv2d(num_channels, 4*num_channels, kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(2)
        self.prelu = nn.PReLU()

    def forward(self, x):
        out = self.conv(x)
        out = self.pixel_shuffle(out)
        out = self.prelu(out)
        return out

class SRResNet(nn.Module):
    def __init__(self, upscale_factor=2, num_channels=64, num_residual_blocks=16):
        super(SRResNet, self).__init__()

        # First convolutional layer
        #nn.Conv2d(number of input channels,number of output channels,kernel size,padding)
        #applied symmetric padding, the size of output is the same as that of input
        self.conv1 = nn.Conv2d(3, num_channels, kernel_size=9, padding=4)

        # Residual blocks
        res_blocks = []
        for _ in range(num_residual_blocks):
            res_blocks.append(ResidualBlock(num_channels))

        self.res_blocks = nn.Sequential(*res_blocks)

        # Second convolutional layer
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)

        # Upscaling layers
        upsample_blocks = []
        for _ in range(int(upscale_factor/2)):
            upsample_blocks.append(UpsampleBlock(num_channels))

        self.upsample = nn.Sequential(*upsample_blocks)

        # Third convolutional layer
        self.conv3 = nn.Conv2d(num_channels, 3, kernel_size=9, padding=4)

    def forward(self, x):
        out = self.conv1(x)
        out = self.res_blocks(out)
        out = self.conv2(out)
        out = self.upsample(out)
        out = self.conv3(out)
        return out



In [ ]:
import torch.optim as optim
import math
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SRResNet()
model.to(device)

def psnr(mse, max_val=1.0):
    if mse == 0:
        return float('inf')
    return 20 * math.log10(max_val / math.sqrt(mse))

num_epochs = 10

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001,weight_decay=1e-4)
train_losses, val_losses = [], []
train_psnrs, val_psnrs = [], []
for epoch in range(num_epochs):
    train_loss = 0.0
    val_loss = 0.0
    train_psnr = 0.0
    val_psnr = 0.0
    
    model.train()
    for inputs, targets in train_dataloader_combined:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        mse_loss = criterion(outputs, targets)
        psnr_loss = psnr(mse_loss.item())

        mse_loss.backward()
        optimizer.step()

        train_loss += mse_loss.item() * inputs.size(0)
        train_psnr += psnr_loss * inputs.size(0)

    train_loss /= len(train_dataloader_combined.dataset)
    train_psnr /= len(train_dataloader_combined.dataset)
    train_losses.append(train_loss)
    train_psnrs.append(train_psnr)

    model.eval()
    with torch.no_grad():
        for inputs, targets in val_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            mse_loss = criterion(outputs, targets)
            psnr_loss = psnr(mse_loss.item())
            val_loss += mse_loss.item() * inputs.size(0)
            val_psnr += psnr_loss * inputs.size(0)
        val_loss /= len(val_dataloader.dataset)
        val_losses.append(val_loss)
        val_psnr /= len(val_dataloader.dataset)
        val_psnrs.append(val_psnr)
    print('Epoch [{}/{}], Train Loss: {:.4f}, Train PSNR: {:.4f}, Val Loss: {:.4f}, Val PSNR: {:.4f}'
          .format(epoch+1, num_epochs, train_loss, train_psnr, val_loss, val_psnr))


PicklingError: Can't pickle <function <lambda> at 0x7f90636823a0>: attribute lookup <lambda> on __main__ failed

In [ ]:
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

plt.plot(train_psnrs, label='Training PSNR')
plt.plot(val_psnrs, label='Validation PSNR')
plt.xlabel('Epoch')
plt.ylabel('PSNR')
plt.title('Training and Validation PSNR')
plt.legend()
plt.show()


In [ ]:
#Test loss
def psnr(mse, max_val=1.0):
    if mse == 0:
        return float('inf')
    return 20 * math.log10(max_val / math.sqrt(mse))

num_epochs = 10
model.eval()
test_loss = 0.0
test_psnr = 0.0
with torch.no_grad():
    for inputs, targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        mse_loss = criterion(outputs, targets)
        psnr_loss = psnr(mse_loss.item())
        test_loss += mse_loss.item() * inputs.size(0)
        test_psnr += psnr_loss * inputs.size(0)
    test_loss /= len(test_dataloader.dataset)
    test_psnr /= len(test_dataloader.dataset)

print('Test Loss: {:.4f}, Test PSNR: {:.4f}'.format(test_loss,test_psnr))


In [ ]:
# Print a few examples of HR image, LR image, and trained 
model.eval()
with torch.no_grad():
    for inputs, targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        break
    inputs = inputs.cpu()
    targets = targets.cpu()
    outputs = outputs.cpu()
    for i in range(3):
        input_img = transforms.ToPILImage()(inputs[i])
        target_img = transforms.ToPILImage()(targets[i])
        output_img = transforms.ToPILImage()(outputs[i])
        plt.subplot(1, 3, 1)
        plt.imshow(input_img)
        plt.title('LR Image')
        plt.axis('off')
        plt.subplot(1, 3, 2)
        plt.imshow(output_img)
        plt.title('Trained Output')
        plt.axis('off')
        plt.subplot(1, 3, 3)
        plt.imshow(target_img)
        plt.title('HR Image')
        plt.axis('off')
        plt.show()